# Experiment 02

This Experiment consists in reproduce the exactly the same experiment as proposed by Martinez (https://www.researchgate.net/publication/221533996_From_an_artificial_neural_network_to_a_stock_market_day-trading_system_A_case_study_on_the_BMF_BOVESPA).

In Summary:
- Each node from ANN Implements a Logistic Function.
- 3 Classic layers:
    - Until 33 Nodes in the Input Layer
    - 2 Node as Outpuy Layers
    - The hidden layers has sqrt(# Input nodes * # Output Nodes) nodes - 6, 7, 8
- 100,000 epochs
- Learning Rate constat and equals to 0.01
- The Momentum from the Back-Propagation Algorithm is 0.8


Change: sliding window

In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5
TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'
TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.04.05'
TIMES = 10

### Auxiliary functions

In [3]:
def Normalize(df):
    date = df['date']
    maxVal = df[['high', 'low']].max()
    normalized = removeColumns(df,['date'])/(removeColumns(df,['date']).max()*2)
    
    df.loc[:, df.columns != 'date'] = normalized
    
    return df, maxVal

### Opening the dataset

In [4]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")

df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

df.tail(290)

,date,open,high,low,close
1049,2020.03.31,13.16,14.07,13.16,13.54
1050,2020.04.01,13.01,14.08,12.89,13.84
1051,2020.04.02,14.90,16.01,14.52,15.01
1052,2020.04.03,15.77,15.83,14.44,14.84
1053,2020.04.06,15.53,15.58,14.69,15.26
...,...,...,...,...,...
1334,2021.05.26,25.91,26.18,25.65,26.09
1335,2021.05.27,26.08,26.25,25.77,25.91
1336,2021.05.28,26.20,27.02,26.15,26.99
1337,2021.05.31,26.96,27.20,26.78,26.87


In [5]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [6]:
##df = getPeriod(df, '2020.03.02','2021.04.30', True)
##initialDf = getPeriod(df, '2020.03.02','2021.04.30', True)

In [7]:
initialDf_PETR4 = getPeriod(df, '2019.01.02','2021.06.01', True)
initialDf_VALE3 = getPeriod(df2, '2019.01.02','2021.06.01', True)

In [8]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2019.01.02,21.18,22.73,20.93,22.60,20.609175,21.499077,20.380957,21.271656,21.776315,...,19.18,20.37,19.55,20.08,19.41,19.69,20.38,20.39,19.50,19.63
1,2019.01.03,22.51,23.32,22.36,23.16,21.242783,22.106051,21.040638,21.901104,23.157151,...,19.99,20.36,19.34,20.37,19.18,20.37,19.55,20.08,19.41,19.69
2,2019.01.04,23.35,23.43,22.99,23.22,21.945189,22.547367,21.690425,22.340736,24.186721,...,20.74,21.31,20.22,20.65,19.99,20.36,19.34,20.37,19.18,20.37
3,2019.01.07,23.35,24.35,23.20,23.59,22.413459,23.148245,22.193617,22.757157,24.644733,...,20.93,22.60,20.77,21.45,20.74,21.31,20.22,20.65,19.99,20.36
4,2019.01.08,23.86,23.88,23.27,23.45,22.895640,23.392163,22.552411,22.988105,24.953473,...,22.36,23.16,21.18,22.73,20.93,22.60,20.77,21.45,20.74,21.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
595,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
596,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
597,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [9]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2019.01.02,45.72,46.96,45.52,46.71,45.965361,46.778539,45.580846,46.324671,46.091838,...,45.16,46.11,45.97,46.85,45.91,46.50,46.39,46.78,45.21,45.54
1,2019.01.03,46.44,46.57,44.25,44.80,46.123574,46.709026,45.137231,45.816447,46.532241,...,45.05,45.25,45.81,46.35,45.16,46.11,45.97,46.85,45.91,46.50
2,2019.01.04,45.55,47.95,45.55,47.71,45.932382,47.122684,45.274820,46.447631,46.581616,...,45.94,46.63,45.80,46.07,45.05,45.25,45.81,46.35,45.16,46.11
3,2019.01.07,48.34,49.05,47.28,47.46,46.734922,47.765123,45.943214,46.785088,48.670273,...,45.52,46.71,45.99,46.81,45.94,46.63,45.80,46.07,45.05,45.25
4,2019.01.08,47.72,48.27,47.19,47.91,47.063281,47.933415,46.358809,47.160058,49.216909,...,44.25,44.80,45.72,46.96,45.52,46.71,45.99,46.81,45.94,46.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
595,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
596,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
597,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experimento 02: Reproducing the Experiment proposed by Martinez

In [10]:
df = initialDf_PETR4
inputColumns = []

#aux, maxVal = Normalize(df)

df_train = getPeriod(df, TRAIN_BEGIN, TRAIN_END)
df_test = getPeriod(df, TEST_BEGIN, TEST_END)

if(inputColumns == []):
    x_train = df_train.drop(['date', 'high', 'low', 'close'], axis=1)
    y_train = df_train[['high','low']]

    x_test = df_test.drop(['date', 'high', 'low', 'close'], axis=1)
    y_test = df_test[['high','low']]
else:
    x_train = df_train[inputColumns]
    y_train = df_train[['high','low']]

    x_test = df_test[inputColumns]
    y_test = df_test[['high','low']]
    
    
scaler = MaxAbsScaler()
scaler.fit(x_train)

scaler.max_abs_ = scaler.max_abs_*2
scaler.scale_ = scaler.scale_*2

scaler.transform(x_train)
scaler.transform(x_test)

#print(scaler.transform(x_train))

MLP = MLPRegressor(hidden_layer_sizes = (8), #sqrt(33*2) according to the article
               momentum = 0.8,
               max_iter = 100000,
               activation = 'logistic',
               learning_rate = 'constant',
               learning_rate_init = 0.01)

MLP.fit(scaler.transform(x_train), y_train)
y_pred = MLP.predict(scaler.transform(x_test))



y_predFix = y_pred
#y_predFix[:,0] = y_predFix[:,0]*maxVal['high']*2
#y_predFix[:,1] = y_predFix[:,1]*maxVal['low']*2

#y_testFix = y_test*maxVal*2
y_testFix = y_test

#print(y_predFix)
#print(y_testFix)


errorMax = mape(y_testFix['high'],y_predFix[:,0])
errorMin = mape(y_testFix['low'],y_predFix[:,1])
printResult(mape(y_testFix['high'],y_predFix[:,0]), 'High')
printResult(mape(y_testFix['low'],y_predFix[:,1]), 'Low')
#print (mae(y_testFix['high'],y_predFix[:,0]))
#print (mae(y_testFix['low'],y_predFix[:,1]))


[[23.58 23.13]
 [23.55 23.15]
 [23.26 22.61]
 [23.01 22.61]
 [23.58 23.01]
 [23.33 22.92]
 [23.78 23.2 ]
 [23.83 23.02]
 [23.24 22.81]
 [24.89 22.77]
 [24.59 23.76]
 [24.19 23.65]
 [23.94 23.51]
 [24.05 23.61]
 [24.17 23.1 ]
 [24.09 23.36]
 [24.29 23.42]
 [23.93 23.28]
 [23.81 23.3 ]
 [23.64 22.88]
 [23.95 23.12]
 [23.85 23.4 ]
 [24.45 23.46]
 [24.98 24.58]
 [25.17 24.32]
 [25.53 24.69]
 [25.1  24.43]
 [26.38 25.55]
 [26.66 26.2 ]
 [26.8  26.27]
 [26.35 25.8 ]
 [26.28 25.72]
 [26.31 25.94]
 [26.52 25.99]
 [26.62 25.78]
 [26.18 25.65]
 [26.25 25.77]
 [27.02 26.15]
 [27.2  26.78]
 [27.7  27.01]]
[[23.93727683 23.12846776]
 [23.93828624 23.12922522]
 [23.86686255 23.05911339]
 [23.58472603 22.78648203]
 [23.62787842 22.82795866]
 [23.77238327 22.96834488]
 [23.88478495 23.07696115]
 [24.09590794 23.28240802]
 [23.95412023 23.14409719]
 [23.83219425 23.02538004]
 [24.44784295 23.62261949]
 [24.57981128 23.74757368]
 [24.47347093 23.64593389]
 [24.48357592 23.65604625]
 [24.43767968 23.6089

In [11]:
print(scaler.max_abs_)
print(scaler.scale_)
print(scaler.transform(x_train))

[60.88       59.22485372 60.12802827 58.31992712 59.07142551 64.65259149
 57.95141677 65.92182771 59.1523401  63.33566302 57.9292009  65.59685429
 57.86163358 60.88       61.46       59.26       60.22       60.88
 61.46       59.26       60.22       60.88       61.46       59.26
 60.22       60.88       61.46       59.26       60.22       60.88
 61.46       59.26       60.22      ]
[60.88       59.22485372 60.12802827 58.31992712 59.07142551 64.65259149
 57.95141677 65.92182771 59.1523401  63.33566302 57.9292009  65.59685429
 57.86163358 60.88       61.46       59.26       60.22       60.88
 61.46       59.26       60.22       60.88       61.46       59.26
 60.22       60.88       61.46       59.26       60.22       60.88
 61.46       59.26       60.22      ]
[[0.21616294 0.21869497 0.22786277 ... 0.21314676 0.20047249 0.21288608]
 [0.21369908 0.21902041 0.22996418 ... 0.23283436 0.20941613 0.22999004]
 [0.24474376 0.22987479 0.24206451 ... 0.23917995 0.22544718 0.23115244]
 ...
 [0.37

In [12]:
scaler.max_abs_ = scaler.max_abs_*2
scaler.scale_ = scaler.scale_*2
print(scaler.max_abs_)
print(scaler.scale_)

print(scaler.transform(x_train))

[121.76       118.44970744 120.25605653 116.63985423 118.14285101
 129.30518297 115.90283353 131.84365542 118.3046802  126.67132605
 115.8584018  131.19370857 115.72326716 121.76       122.92
 118.52       120.44       121.76       122.92       118.52
 120.44       121.76       122.92       118.52       120.44
 121.76       122.92       118.52       120.44       121.76
 122.92       118.52       120.44      ]
[121.76       118.44970744 120.25605653 116.63985423 118.14285101
 129.30518297 115.90283353 131.84365542 118.3046802  126.67132605
 115.8584018  131.19370857 115.72326716 121.76       122.92
 118.52       120.44       121.76       122.92       118.52
 120.44       121.76       122.92       118.52       120.44
 121.76       122.92       118.52       120.44       121.76
 122.92       118.52       120.44      ]
[[0.10808147 0.10934748 0.11393138 ... 0.10657338 0.10023625 0.10644304]
 [0.10684954 0.1095102  0.11498209 ... 0.11641718 0.10470807 0.11499502]
 [0.12237188 0.1149374  0.12

In [58]:
max_test = np.array(y_testFix['high'])
min_test = np.array(y_testFix['low'])

max_pred = np.array(y_predFix[:,0])
min_Pred  = np.array(y_predFix[:,1])

print(mape(max_test, max_pred))



#print(np.array(y_testFix))
#print(np.array(y_predFix))
#print(np.append(np.array(y_testFix)[:0],np.array(y_predFix)[:0]))

#errorMax = mape(y_testFix['high'],y_predFix[:,0])
#errorMin = mape(y_testFix['low'],y_predFix[:,1])

(1.327276129518633, 0.8887437651323804)
